## NBA Prediction
--------------
**Author: Collins Emezie Ugwuozor** . [Linkedin](https://www.linkedin.com/in/collins-ugwuozor)
. [Github](https://github.com/ClassicCollins)
. [Twitter](https://x.com/ClassicCollins2)
. [Email](mailto:ugwuozorcollinsemezie@gmail.com)
. [WhatsApp](https://wa.me/+2348037953669)

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load Elo ratings data
# elo_data = pd.read_csv('https://github.com/fivethirtyeight/data/blob/master/nba-elo/nbaallelo.csv?raw=true')
# modern_raptor_players =  pd.read_csv('https://github.com/fivethirtyeight/data/blob/master/nba-raptor/modern_RAPTOR_by_player.csv?raw=true')
# modern_raptor_team =  pd.read_csv('https://github.com/fivethirtyeight/data/blob/master/nba-raptor/modern_RAPTOR_by_team.csv?raw=true')
# win_prob = pd.read_csv('https://github.com/fivethirtyeight/data/blob/master/nba-winprobs/nba.tsv?raw=true')
# team_historical_raptor_data = pd.read_csv('https://github.com/fivethirtyeight/data/blob/master/nba-raptor/historical_RAPTOR_by_team.csv?raw=true')
# player_historical_raptor_data = pd.read_csv('https://github.com/fivethirtyeight/data/blob/master/nba-raptor/historical_RAPTOR_by_player.csv?raw=true')

In [2]:
# Load data
elo_data = pd.read_csv('elo_data.csv')
nba_elo = pd.read_csv('nba_elo.csv')
nba_elo_latest = pd.read_csv('nba_elo_latest.csv')
modern_raptors_team = pd.read_csv('morden_raptors_team.csv')
modern_raptors_player = pd.read_csv('morden_raptors_players.csv')
raptor_data = pd.read_csv('raptor_data.csv')
raptor_data_team = pd.read_csv('raptor_data_team.csv')
win_prob = pd.read_csv('win_prob.tsv', sep='\t')
data_definition = pd.read_csv('data_definition.csv')
team_historical_raptor_data = pd.read_csv('team_historical_raptor_data.csv')
player_historical_raptor_data = pd.read_csv('player_historical_raptor_data.csv')

In [3]:


# Load data
table1 = pd.read_csv("nba_elo.csv")
table2 = pd.read_csv("nba_elo_latest.csv")

def preprocess_data(table1, table2):
    table1['date'] = pd.to_datetime(table1['date'])
    df1 = table1[(table1['season'] >= 2019) & (table1['season'] <= 2022)]
    
    table2['date'] = pd.to_datetime(table2['date'])
    df2 = table2[table2['season'] == 2023]
    
    # Determine game results (1 if team1 wins, 0 if team2 wins)
    df1['game_result'] = (df1['score1'] > df1['score2']).astype(int)
    df2['game_result'] = (df2['score1'] > df2['score2']).astype(int)
    
    # Drop insignificant columns
    cols_to_drop = ['date', 'season', 'playoff', 'team1', 'team2', 'elo1_post', 'elo2_post', 'neutral', 
                    'carm-elo1_pre', 'carm-elo2_pre', 'carm-elo_prob1', 'carm-elo_prob2', 
                    'carm-elo1_post', 'carm-elo2_post', 'importance', 'total_rating', 'score1', 'score2']
    df1.drop(columns=cols_to_drop, inplace=True)
    df2.drop(columns=cols_to_drop, inplace=True)
    
    df1.reset_index(drop=True, inplace=True)
    
    train = df1
    test = df2
    
    return train, test

train, test = preprocess_data(table1, table2)


C:\Users\collins\AppData\Local\Temp\ipykernel_18044\1991373174.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['game_result'] = (df1['score1'] > df1['score2']).astype(int)
C:\Users\collins\AppData\Local\Temp\ipykernel_18044\1991373174.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop(columns=cols_to_drop, inplace=True)


In [4]:

def build_model(train):
    y = train.pop('game_result')
    
    # Train/Test Split
    X_train, X_val, y_train, y_val = train_test_split(train, y, test_size=0.2, random_state=17)
    
    # Model Training
    GBoost_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=2, random_state=42)
    GBoost_model.fit(X_train, y_train)
    
    # Make predictions on the validation set
    y_gboost_pred = GBoost_model.predict(X_val)
    
    # Evaluate the model's accuracy
    accuracy_gb = accuracy_score(y_val, y_gboost_pred)
    print("Gradient Boosting Model Accuracy: ", accuracy_gb)
    
    # Train Random Forest model
    RF_model = RandomForestClassifier(n_estimators=100, random_state=42)
    RF_model.fit(X_train, y_train)
    
    # Make predictions on the validation set
    y_rf_pred = RF_model.predict(X_val)
    
    # Evaluate the model's accuracy
    accuracy_rf = accuracy_score(y_val, y_rf_pred)
    print("Random Forest Model Accuracy: ", accuracy_rf)
    
    # Train the logistic regression model
    lr_model = LogisticRegression(max_iter=10000)
    lr_model.fit(X_train, y_train)
    
    # Make predictions on the validation set
    y_lr_pred = lr_model.predict(X_val)
    accuracy_lr = accuracy_score(y_val, y_lr_pred)
    print("Logistic Regression Model Accuracy: ", accuracy_lr)
    
    return GBoost_model, RF_model, lr_model


In [5]:

GBoost_model, RF_model, lr_model = build_model(train)


Gradient Boosting Model Accuracy:  0.6303030303030303
Random Forest Model Accuracy:  0.6272727272727273
Logistic Regression Model Accuracy:  0.6676767676767676


In [6]:

# Save the models
joblib.dump(lr_model, 'nba_lr_model.pkl')
joblib.dump(GBoost_model, 'nba_GBoost_model.pkl')
joblib.dump(RF_model, 'nba_rf_model.pkl')
print("Models saved as nba_lr_model.pkl, nba_rf_model.pkl and nba_GBoost_model.pkl")


Models saved as nba_lr_model.pkl, nba_rf_model.pkl and nba_GBoost_model.pkl


In [7]:

def evaluate_model(test):
    test.drop(columns=['game_result'], inplace=True)
    filetime = datetime.now().strftime('%Y%m%d-%H%M')
    print(filetime)
    
    result = pd.DataFrame({'game_predict': lr_model.predict(test)}).sort_index()
    print(result.head())
    
    return result

result = evaluate_model(test)


20241029-1846
   game_predict
0             1
1             1
2             1
3             1
4             1


In [8]:
# Preprocess the 2023 season data
season_2023 = table2.copy()
season_2023.drop(columns=['neutral', 'carm-elo1_pre', 'carm-elo2_pre', 'carm-elo_prob1', 'carm-elo_prob2',
                          'date', 'season', 'carm-elo1_post', 'carm-elo2_post', 'elo1_post', 'elo2_post',
                          'raptor1_pre', 'raptor2_pre', 'raptor_prob1', 'raptor_prob2', 'importance', 'total_rating'], inplace=True)

season_2023_predicted = pd.concat([season_2023, result], axis=1).reindex(season_2023.index)
season_2023_played_qualified_prediction = season_2023_predicted[season_2023_predicted['playoff'].isna()]


In [9]:

# Define function to process and save results for each division
def process_division(df, teams, division_name):
    home_games = df[df['team1'].isin(teams)]
    home_away_games = home_games[home_games['team2'].isin(teams)]
    division_results = home_away_games.groupby(by=['team1']).sum().sort_values(by=['game_predict', 'quality'], ascending=[False, False])
    division_results.reset_index(inplace=True)
    division_results.to_csv(f'results/{division_name}.csv')
    return division_results


In [10]:

# Process each division
Atlantic_teams = ['BOS', 'BRK', 'NYK', 'PHI', 'TOR']
Central_teams = ['CHI', 'CLE', 'DET', 'IND', 'MIL']
Southeast_teams = ['ATL', 'CHO', 'MIA', 'ORL', 'WAS']
Northwest_teams = ['DEN', 'MIN', 'OKC', 'POR', 'UTA']
Pacific_teams = ['GSW', 'LAC', 'LAL', 'PHO', 'SAC']
Southwest_teams = ['DAL', 'HOU', 'MEM', 'NOP', 'SAS']

Atlantic = process_division(season_2023_played_qualified_prediction, Atlantic_teams, 'Atlantic')
Central = process_division(season_2023_played_qualified_prediction, Central_teams, 'Central')
Southeast = process_division(season_2023_played_qualified_prediction, Southeast_teams, 'Southeast')
Northwest = process_division(season_2023_played_qualified_prediction, Northwest_teams, 'Northwest')
Pacific = process_division(season_2023_played_qualified_prediction, Pacific_teams, 'Pacific')
Southwest = process_division(season_2023_played_qualified_prediction, Southwest_teams, 'Southwest')


In [11]:

# Process conference results
Eastern_conference_teams = [Atlantic.team1[0], Central.team1[0], Southeast.team1[0]]
Western_conference_teams = [Northwest.team1[0], Pacific.team1[0], Southwest.team1[0]]

Eastern_conference_home = season_2023_played_qualified_prediction[season_2023_played_qualified_prediction['team1'].isin(Eastern_conference_teams)]
Eastern_conference_home_away = Eastern_conference_home[Eastern_conference_home['team2'].isin(Eastern_conference_teams)]
Eastern_conference = Eastern_conference_home_away.groupby(by=['team1']).sum().sort_values(by=['game_predict', 'quality'], ascending=[False, False])
Eastern_conference.reset_index(inplace=True)
Eastern_conference.to_csv('results/Eastern_conference.csv')

Western_conference_home = season_2023_played_qualified_prediction[season_2023_played_qualified_prediction['team1'].isin(Western_conference_teams)]
Western_conference_home_away = Western_conference_home[Western_conference_home['team2'].isin(Western_conference_teams)]
Western_conference = Western_conference_home_away.groupby(by=['team1']).sum().sort_values(by=['game_predict', 'quality'], ascending=[False, False])
Western_conference.reset_index(inplace=True)
Western_conference.to_csv('results/Western_conference.csv')


In [13]:

# Process NBA Finals
nba_Semi_finals = season_2023_played_qualified_prediction[(season_2023_played_qualified_prediction['team1'] == Eastern_conference.team1[0]) |
                                                          (season_2023_played_qualified_prediction['team1'] == Western_conference.team1[0])]
nba_finals = nba_Semi_finals[(nba_Semi_finals['team2'] == Western_conference.team1[0]) |
                             (nba_Semi_finals['team2'] == Eastern_conference.team1[0])]
nba_finals = nba_finals.groupby(by=['team1']).sum().sort_values(by='game_predict', ascending=False)
nba_finals.reset_index(inplace=True)

winner = nba_finals.head(1)
winner.to_csv('results/winner.csv')
print(winner.team1)

0    DEN
Name: team1, dtype: object
